# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [2]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.core.debugger import set_trace




## Env setup


In [3]:
# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")


## Object detection imports
 Here are the imports from the object detection module.


In [4]:
from utils import label_map_util

from utils import visualization_utils as vis_util

## Model preparation 

### Variables

 Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  
 
 By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.


In [5]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

## Download Model


In [6]:
if ~(os.path.isfile(MODEL_FILE)):
    opener = urllib.request.URLopener()
    opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
    tar_file = tarfile.open(MODEL_FILE)
    for file in tar_file.getmembers():
      file_name = os.path.basename(file.name)
      if 'frozen_inference_graph.pb' in file_name:
        tar_file.extract(file, os.getcwd())


## Load a (frozen) Tensorflow model into memory.


In [7]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine


In [8]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [9]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)


## Detection

In [10]:
# CELL NOT NEEDED 
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'test_images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(5, 6) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [ ]:
cap=cv2.VideoCapture("videos/vid.mp4")
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        detection = 1
        min_tresh = .5
        # tracker parameters
        lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
        while cap.isOpened():

            ret,image_np = cap.read()
            if ret == False:
                break;



            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            
            gray = cv2.cvtColor(image_np,cv2.COLOR_BGR2GRAY)
            # Actual detection.
            if detection == 1:
                (boxes, scores, classes, num) = sess.run(
                [detection_boxes, detection_scores, detection_classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
                boxes,scores,classes,detect = vis_util.findBoxes(
                    np.squeeze(boxes),
                    np.squeeze(classes).astype(np.int32),
                    np.squeeze(scores),
                    min_tresh)
                #print(detect)
                if detect:
                    #print (scores.shape)
                    detection = 0
            else:
                
                ind_pt = 0
                new_imp_pts = []
                for item in imp_pts:
                    p1, st, err = cv2.calcOpticalFlowPyrLK(old_frame, gray, item, None, **lk_params)
                    good_new = p1[st==1]
                    good_old = item[st==1]
                    #set_trace()
                    
                    
                    dist_pts = np.linalg.norm(good_new-good_old,axis=1)
                    dist_mean = np.mean(dist_pts)
                    
                    if len(good_new) == 0:
                        detection = 1
                        continue

                    
                    good_dist_pts = dist_pts>dist_mean/2
                    good_new = good_new[good_dist_pts]
                    l,t=np.min(good_new,axis=0)
                    r,b=np.max(good_new,axis=0)
                    boxes[ind_pt] = [t,l,b,r]
                    ind_pt+=1
                    #set_trace()
                    new_imp_pts.append(good_new.reshape(-1,1,2))
                imp_pts = new_imp_pts    
                #tracking
            # Visualization of the results of a detection.
            #print (len(boxes))
            #print (scores)
            #print (boxes)
            if detection == 0:
                temp,box_abs = vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                boxes,
                classes.astype(np.int32),
                scores,
                category_index,
                use_normalized_coordinates=True,
                line_thickness=8)
            elif detection !=1:
                temp,box_abs = vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                boxes,
                classes.astype(np.int32),
                scores,
                category_index,
                use_normalized_coordinates=False,
                line_thickness=8)
                
            
            if (detection == 0):
                sift = cv2.xfeatures2d.SIFT_create()
                kp, des = sift.detectAndCompute(gray,None)
                pts= [kp[idx].pt for idx in range(len(kp))]
                imp_pts = []
                for box in box_abs:
                    temp=[]
                    l,t,r,b = tuple(box)
                    for pt in pts:
                        if pt[0]>l and pt[0]<r and pt[1]>t and pt[1]<b:
                            temp.append(pt)
                    imp_pts.append(np.array(temp,dtype=np.float32).reshape(-1,1,2))
                detection = 2
            old_frame = gray
                
            cv2.imshow('object detection',image_np)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break


    cap.release()
    cv2.destroyAllWindows()



In [ ]:
imp_pts

In [ ]:
pts


In [ ]:
box_abs


In [ ]:
boc = box_abs[0]


In [ ]:
l,t,r,b = tuple(boc)


In [ ]:
ae=[]
for pt in pts:
    if pt[0]>l and pt[0]<r and pt[1]>t and pt[1]<b:
        ae.append(pt)

In [ ]:
len(pts)

In [ ]:
i=[i for i,v in enumerate(a) if v > 5.5]

In [ ]:
i

In [ ]:
t=a[i]

In [ ]:
t.shape[0] == 0

In [ ]:
classes

In [ ]:
vis_util.findBoxes()

In [ ]:
from vis_util import findBoxes

In [ ]:
from utils.visualization_utils import findBoxes

In [ ]:
vis_util.findBoxes()

In [ ]:
classes

In [ ]:
boxes.shape

In [ ]:
aaa


In [ ]:
aaa.append(1)